In [13]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from datetime import datetime
import time

In [ ]:

# Alpaca API credentials
ALPACA_API_KEY
ALPACA_SECRET_KEY

In [15]:

client = StockHistoricalDataClient(api_key=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY)

symbol_list = [
    # Large-Cap Blue Chips (Stable)
    "AAPL", "MSFT", "JNJ", "PG", "JPM",
    
    # Growth Stocks (Higher Volatility)
    "NVDA", "AMD", "TSLA", "CRM", "AMZN",
    
    # Value/Cyclical Stocks
    "CAT", "CVX", "BA", "FCX", "GS",
    
    # Defensive Stocks
    "KO", "WMT", "PFE", "VZ", "NEE",
    
    # Market ETFs (Benchmarks)
    "SPY", "QQQ", "IWM", "XLF", "XLE"
]


for symbol in symbol_list:
    request_params = StockBarsRequest(
                    symbol_or_symbols=symbol,
                    timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),
                    start=datetime(2015, 1, 1),
                    adjustment="all"
                )
    bars = client.get_stock_bars(request_params)
    # Reset index to make timestamp a column
    stock_data = bars.df.reset_index()
    # Add symbol column
    stock_data['symbol'] = symbol
    # Convert timestamp to desired format
    stock_data['time'] = stock_data['timestamp'].dt.strftime('%Y-%m-%d')
    # Reorder columns and drop the original timestamp column
    stock_data = stock_data[['symbol', 'time', 'open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap']]
    # make all the columns start with uppercase
    stock_data.columns = [col.capitalize() for col in stock_data.columns]
    output_filename = f"{symbol}_D1_raw_data.csv"
    stock_data.to_csv(output_filename, index=False)
    # wait for 1 second to avoid rate limiting
    time.sleep(2)